In [18]:
%%capture
%run full_setup.py

In [19]:
from tfidf_corpus_dictionary import get_tfidf_tokendocs_corpus_dict

from gensim.models import LdaModel, LsiModel
from sklearn.decomposition import NMF, PCA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection

With this function we get various objects needed for modelling:
1. TFIDF matrix as input data, with specified parameters
2. feature names as the words retained with TFIDF
3. tokenized documents, a list of lists, where the inner lists contain the tokens for each document
4. corpus, gensim object needed for modelling with that package
4. dictionary, gensim object containing informations on the words of the corpus and their positions

In [20]:
tfidf_matrix, feature_names, tokenized_docs, corpus, dictionary = get_tfidf_tokendocs_corpus_dict(df, max_df=0.5, min_df=5, max_features=5000)

Now we'll evaluate different topic models based on Coherence score.

Coherence is a metric used to evalute topics quality. The higher the coherence, the better the model did in creating the topics.

For every model we'll use a function to retrieve for different numbers of topics the coherence (5, 10, 15, 20, 50). This information will be used to evaluate how the models performed as the number of topics changes

In [21]:
from coherence_topics import coherence_topics

In [22]:
evaluation = dict()
models = ['LDA', 'LSA', 'NMF']

for mod in models:
    metrics = coherence_topics(model_name=mod, corpus=corpus, dictionary=dictionary,
                               texts=tokenized_docs, feature_names=feature_names, tfidf=tfidf_matrix)
    evaluation[mod] = metrics

c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\linalg\_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\linalg\_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\li

In [23]:
evaluation['LDA']

[(5, 0.545979304624336),
 (10, 0.5050314716993893),
 (15, 0.5285162980465483),
 (20, 0.5134451552900542),
 (50, 0.4630950783156206)]

In [24]:
evaluation['LSA']

[(5, 0.6304194600581545),
 (10, 0.4839449986875244),
 (15, 0.4017113657264382),
 (20, 0.41826946770145207),
 (50, 0.35332142182573034)]

In [25]:
evaluation['NMF']

[(5, 0.3597199653853072),
 (10, 0.35971996538530726),
 (15, 0.35971996538530715),
 (20, 0.35971996538530715),
 (50, 0.35971996538530715)]

Now we'll fit the LDA model with the number of topics that yields the highest coherence

In [31]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5,
                     alpha='symmetric', eta='auto', passes=5, random_state=1)

In [33]:
for topic in lda_model.print_topics(num_words=15):
    topic_index, words = topic
    word_list = [word.split("*")[1].strip().strip('"') for word in words.split(" + ")]
    print(f"Topic {topic_index}: {', '.join(word_list)}")

Topic 0: car, drive, bike, sale, buy, speed, sell, price, distribution, game, motorcycle, usa, scsi, power, work
Topic 1: window, file, email, system, key, program, chip, computer, help, phone, software, version, work, problem, card
Topic 2: god, christian, jesus, bible, christ, religion, morality, christianity, church, faith, objective, believe, sin, truth, belief
Topic 3: beast, anthony, walker, mask, amanda, duo, murphy, pt, intercon, hamburg, singapore, cs, alex, vesselin, stats
Topic 4: government, state, year, israel, israeli, armenian, law, writes, kill, world, gun, child, case, david, post


In [27]:
import pyLDAvis, pyLDAvis.gensim

In [34]:
# Visualize the LDA model using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='tsne')
pyLDAvis.display(vis)

Now we're going to do the same for LSA

In [29]:
lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=5)

c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\linalg\_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\linalg\_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\li

In [30]:
for topic in lsi_model.print_topics():
    topic_index, words = topic
    word_list = [word.split("*")[1].strip().strip('"') for word in words.split(" + ")]
    print(f"Topic {topic_index}: {', '.join(word_list)}")

Topic 0: system, work, window, god, problem, year, drive, distribution, file, computer
Topic 1: god, window, file, card, drive, christian, jesus, driver, program, believe
Topic 2: game, god, team, window, player, file, play, year, win, christian
Topic 3: key, chip, god, clipper, encryption, government, window, game, escrow, jesus
Topic 4: drive, window, file, car, scsi, game, team, program, key, ide
